Wikipedia IT企業5社の重要単語を求める  
企業 サイバーエージェント, メルカリ, Google, グノシー, DeNA

記事を収集

クローラ

In [91]:
from bs4 import BeautifulSoup
import urllib3
import certifi
import lxml
import time
from tqdm import tqdm
import csv
import re

In [4]:
def get_soup(url):
    http = urllib3.PoolManager(
        cert_reqs='CERT_REQUIRED',
        ca_certs=certifi.where())
    r = http.request('GET', url)
    return BeautifulSoup(r.data, 'lxml')

In [7]:
soup_ca = get_soup('https://ja.wikipedia.org/wiki/%E3%82%B5%E3%82%A4%E3%83%90%E3%83%BC%E3%82%A8%E3%83%BC%E3%82%B8%E3%82%A7%E3%83%B3%E3%83%88')
soup_gn = get_soup('https://ja.wikipedia.org/wiki/%E3%82%B0%E3%83%8E%E3%82%B7%E3%83%BC')
soup_gg = get_soup('https://ja.wikipedia.org/wiki/Google')
soup_dn = get_soup('https://ja.wikipedia.org/wiki/%E3%83%87%E3%82%A3%E3%83%BC%E3%83%BB%E3%82%A8%E3%83%8C%E3%83%BB%E3%82%A8%E3%83%BC')
soup_mr = get_soup('https://ja.wikipedia.org/wiki/%E3%83%A1%E3%83%AB%E3%82%AB%E3%83%AA')

本文日本語抽出

In [206]:
def extract_jp_from_soup(soup):
    return re.sub(r'[^あ-ん ｱ-ﾝ ア-ン 一-龥 ー ]', '', str(soup))

def remove_docname(name, text):
    return text.replace(name,'')

text_ca = extract_jp_from_soup(soup_ca)
text_gn = extract_jp_from_soup(soup_gn)
text_gg = extract_jp_from_soup(soup_gg)
text_dn = extract_jp_from_soup(soup_dn)
text_mr = extract_jp_from_soup(soup_mr)

text_ca = remove_docname('サイバーエージェント',text_ca)
text_gn = remove_docname('グノシー',text_gn)
text_gg = remove_docname('グーグル',text_gg)
text_dn = remove_docname('ディー・エヌ・エー',text_dn)
text_mr = remove_docname('メルカリ',text_mr)

MECAB

In [88]:
import MeCab

In [207]:
mecab = MeCab.Tagger('-Owakati')

In [208]:
parsed_text_ca = mecab.parse(text_ca)
parsed_text_gn = mecab.parse(text_gn)
parsed_text_gg = mecab.parse(text_gg)
parsed_text_dn = mecab.parse(text_dn)
parsed_text_mr = mecab.parse(text_mr)

TFIDF

In [160]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [225]:
vectorizer = TfidfVectorizer(min_df=1, max_df=3)

In [226]:
corpus = [parsed_text_ca,parsed_text_gn,parsed_text_gg,parsed_text_dn,parsed_text_mr]
X = vectorizer.fit_transform(corpus)

In [227]:
terms = vectorizer.get_feature_names()

In [228]:
tfidfs = X.toarray()

In [229]:
tfidfs.shape

(5, 3818)

データフレーム

In [176]:
import pandas as pd
from IPython.display import display

In [230]:
df = pd.DataFrame({'terms':terms, 'サイバーエージェント': tfidfs[0], 'グノシー': tfidfs[1], 'グーグル':tfidfs[2], 'ディー・エヌ・エー':tfidfs[3], 'メルカリ':tfidfs[4]})

In [231]:
def display_top_k_words(doc, k, df):
    df = df.sort_values(doc, ascending=False)
    display(doc, df['terms'].iloc[:k])
display_top_k_words('サイバーエージェント', 5, df)
display_top_k_words('グノシー', 5, df)
display_top_k_words('グーグル', 5, df)
display_top_k_words('ディー・エヌ・エー', 5, df)
display_top_k_words('メルカリ', 5, df)


'サイバーエージェント'

2131    合併
2444    平成
2162    吸収
2333    学校
2031    動画
Name: terms, dtype: object

'グノシー'

2857        東洋
2200        四季
2881        業績
946     ダイヤモンド
3196        福島
Name: terms, dtype: object

'グーグル'

3526     買収
1056     ドル
2698     政府
3348     英語
3496    識別子
Name: terms, dtype: object

'ディー・エヌ・エー'

2060          南場
2333          学校
2897    横浜ベイスターズ
2895          横浜
2780          智子
Name: terms, dtype: object

'メルカリ'

1954        出品
2185        商品
3054        現金
1236    フリマアプリ
1028      トラブル
Name: terms, dtype: object